In [10]:
from examples.seismic import Receiver, Model

from examples.seismic.acoustic import acoustic_setup
from examples.seismic.vector_reflectivity import vector_reflectivity_setup
from examples.seismic.variable_density import variable_density_setup

import numpy as np

from devito import norm, configuration, VectorTimeFunction, TimeFunction

import matplotlib.pyplot as plt
from sympy import init_printing, latex
from IPython.display import Latex, Markdown

configuration['ignore-unknowns'] = 1

In [11]:
init_printing(scale=2, fontsize='12pt')

In [12]:
nx = 20
nz = 25

shape = (nx, nz)

presets = {
    'layers-viscoacoustic': {'preset': 'layers-viscoacoustic', 'nlayers': 2, 'shape': shape},
    'layers-isotropic': {'preset': 'layers-isotropic', 'nlayers': 2, 'shape': shape},
    'layers-isotropic-constant-density': {'preset': 'layers-isotropic', 'nlayers': 2, 'shape': shape, 'b': np.ones(shape)},
}

### Acoustic

In [13]:
tn = 500.  # Final time

# Create solver from preset
solver = acoustic_setup(spacing=[15. for _ in shape],
                        nbl=10, tn=tn, space_order=2, time_order=1,
                        **(presets['layers-isotropic']), dtype=np.float64)

# Create adjoint receiver symbol
srca = Receiver(name='srca', grid=solver.model.grid,
                time_range=solver.geometry.time_axis,
                coordinates=solver.geometry.src_positions)

# Run forward and adjoint operators
rec = solver.forward(save=False)[0]
solver.adjoint(rec=rec, srca=srca)

# Adjoint test: Verify <Ax,y> matches  <x, A^Ty> closely
term1 = np.dot(srca.data.reshape(-1), solver.geometry.src.data)
term2 = norm(rec) ** 2
print('<x, A*y>: %f, <Ax,y>: %f, difference: %4.4e, ratio: %f'
    % (term1, term2, (term1 - term2)/term1, term1/term2))


assert np.isclose((term1 - term2)/term1, 0., atol=1.e-11)

Operator `initdamp` ran in 0.01 s
Operator `Forward` ran in 0.01 s
No source type defined, returning uninitiallized (zero) source
Operator `Adjoint` ran in 0.01 s


<x, A*y>: 272611.466153, <Ax,y>: 272611.466153, difference: -2.1352e-16, ratio: 1.000000


/tmp/ipykernel_16146/540016980.py:20: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('<x, A*y>: %f, <Ax,y>: %f, difference: %4.4e, ratio: %f'


In [14]:
Markdown(
    """
$\\langle x, A^Ty \\rangle$ | $\\langle Ax,y\\rangle$ | Relative Error | Ratio
---------|--------|----------------|-------
%f | %f | %f | %f
    """ % (term1, term2, (term1 - term2)/term1, term1/term2)
)

/tmp/ipykernel_16146/3318135832.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  """



$\langle x, A^Ty \rangle$ | $\langle Ax,y\rangle$ | Relative Error | Ratio
---------|--------|----------------|-------
272611.466153 | 272611.466153 | -0.000000 | 1.000000
    

### Vector Reflectivity

In [15]:
tn = 500.  # Final time

solver = vector_reflectivity_setup(spacing=[15. for _ in shape],
                        nbl=10, tn=tn, space_order=2, time_order=1,
                        **(presets['layers-isotropic-constant-density']), dtype=np.float64, f0=0.010)
                        # **(presets['layers-viscoacoustic']), dtype=np.float64, f0=0.010)

# Create adjoint receiver symbol
srca = Receiver(name='srca', grid=solver.model.grid,
                time_range=solver.geometry.time_axis,
                coordinates=solver.geometry.src_positions)

# Run forward and adjoint operators
rec = solver.forward(save=False)[0]
solver.adjoint(rec=rec, srca=srca)

# Adjoint test: Verify <Ax,y> matches  <x, A^Ty> closely
term1 = np.dot(srca.data.reshape(-1), solver.geometry.src.data)
term2 = norm(rec) ** 2

print('<x, A*y>: %f, <Ax,y>: %f, difference: %4.4e, ratio: %f'
    % (term1, term2, (term1 - term2)/term1, term1/term2))


assert np.isclose((term1 - term2)/term1, 0., atol=1.e-11)

Operator `initdamp` ran in 0.01 s
Operator `Forward` ran in 0.01 s
No source type defined, returning uninitiallized (zero) source
Operator `Adjoint` ran in 0.02 s


<x, A*y>: 283018.393546, <Ax,y>: 283018.393546, difference: -1.6453e-15, ratio: 1.000000


/tmp/ipykernel_16146/3579955609.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('<x, A*y>: %f, <Ax,y>: %f, difference: %4.4e, ratio: %f'


In [16]:
Markdown(
    """
$\\langle x, A^Ty \\rangle$ | $\\langle Ax,y\\rangle$ | Relative Error | Ratio
---------|--------|----------------|-------
%f | %f | %f | %f
    """ % (term1, term2, (term1 - term2)/term1, term1/term2)
)

/tmp/ipykernel_16146/3318135832.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  """



$\langle x, A^Ty \rangle$ | $\langle Ax,y\rangle$ | Relative Error | Ratio
---------|--------|----------------|-------
283018.393546 | 283018.393546 | -0.000000 | 1.000000
    

In [17]:
def latex_print(symbol):
    return Latex(f'$${latex(symbol)}$$'.replace('frac', 'dfrac'))

### Variable Density

In [ ]:
tn = 500.  # Final time

solver = variable_density_setup(spacing=[15. for _ in shape],
                        nbl=10, tn=tn, space_order=2, time_order=1,
                        **(presets['layers-isotropic-constant-density']), dtype=np.float64, f0=0.010)
                        # **(presets['layers-viscoacoustic']), dtype=np.float64, f0=0.010)

# Create adjoint receiver symbol
srca = Receiver(name='srca', grid=solver.model.grid,
                time_range=solver.geometry.time_axis,
                coordinates=solver.geometry.src_positions)

# Run forward and adjoint operators
rec, u, _ = solver.forward(save=False)
solver.adjoint(rec=rec, srca=srca)

# Adjoint test: Verify <Ax,y> matches  <x, A^Ty> closely
term1 = np.dot(srca.data.reshape(-1), solver.geometry.src.data)
term2 = norm(rec) ** 2
print('<x, A*y>: %f, <Ax,y>: %f, difference: %4.4e, ratio: %f'
    % (term1, term2, (term1 - term2)/term1, term1/term2))


assert np.isclose((term1 - term2)/term1, 0., atol=1.e-11)

Operator `initdamp` ran in 0.01 s
Re-initializing damp profile from mask to damp
Model has to be created with `bcs="damp"`for this WaveSolver
Operator `initdamp` ran in 0.01 s
Operator `Forward` ran in 0.01 s
No source type defined, returning uninitiallized (zero) source
Operator `Adjoint` ran in 0.01 s


<x, A*y>: 276482.795970, <Ax,y>: 276482.795970, difference: -1.6842e-15, ratio: 1.000000


/tmp/ipykernel_29801/4089954036.py:20: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('<x, A*y>: %f, <Ax,y>: %f, difference: %4.4e, ratio: %f'


In [ ]:
Markdown(
    """
$\\langle x, A^Ty \\rangle$ | $\\langle Ax,y\\rangle$ | Relative Error | Ratio
---------|--------|----------------|-------
%f | %f | %f | %f
    """ % (term1, term2, (term1 - term2)/term1, term1/term2)
)

/tmp/ipykernel_29801/3318135832.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  """



$\langle x, A^Ty \rangle$ | $\langle Ax,y\rangle$ | Relative Error | Ratio
---------|--------|----------------|-------
276482.795970 | 276482.795970 | -0.000000 | 1.000000
    